In [357]:
#PATH_data = f"../abyss_data/"
#Mount external drive: sudo mount -t drvfs D: /mnt/d
PATH_data = f"/mnt/d/abyss_data/"
PATH_GENO = f"{PATH_data}genotype"
PATH_SUMSTAT = f"{PATH_GENO}/sumstat/"
PATH_SUMSTAT_no_corr = f"{PATH_data}sumstat_no_corr"
PATH_GENO_segmented = f"{PATH_GENO}/segmented/"
PATH_GENO_mini_dim = f"{PATH_GENO}/mini_dim/"
PATH_GENO_maf = f"{PATH_GENO}/maf/"


PATH_PHENO = f"{PATH_data}phenotype/"
PATH_PHENO_ancestry = f"{PATH_PHENO}/ancestry/all.panel"
PATH_PLOTS = f"{PATH_data}plots/"
PATH_PARAMS = './hyper_params_to_test_AF.txt'

In [358]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
from collections import Counter
import os
from os import listdir
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

In [359]:
effect = 1
chrom = 21
sigmap = 3

# If you happen to have labelled population data, we can create a synthetic phenotype containing population stratification

path_humans = PATH_PHENO_ancestry
humans = pd.read_csv(path_humans, sep="\t")
humans = humans.drop(columns=['Unnamed: 5', 'Unnamed: 6'])
humans = humans.rename({'sample': 'FID', 'sample.1': 'IID'}, axis=1)
humans['ancestry'] = humans['super_pop']
humans['ancestry'].mask(humans['super_pop'] == 'AFR', 0, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'AMR', 1, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'EAS', 2, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'EUR', 3, inplace=True)
humans['ancestry'].mask(humans['super_pop'] == 'SAS', 4, inplace=True)

# Make synthetic phenotype with no population stratification and no causal effect

afr = humans.loc[humans['ancestry'] == 0]
amr = humans.loc[humans['ancestry'] == 1]
eas = humans.loc[humans['ancestry'] == 2]
eur = humans.loc[humans['ancestry'] == 3]
sas = humans.loc[humans['ancestry'] == 4]

mu, sigma = 150, sigmap  # mean and standard deviation for africans
afr["no_strat_no_causal"] = np.random.normal(mu, sigma, len(afr))
afr["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for americans
amr["no_strat_no_causal"] = np.random.normal(mu, sigma, len(amr))
amr["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for east asians
eas["no_strat_no_causal"] = np.random.normal(mu, sigma, len(eas))
eas["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for europeans
eur["no_strat_no_causal"] = np.random.normal(mu, sigma, len(eur))  # 4
eur["bias_no_strat"] = 150
mu, sigma = 150, sigmap  # mean and standard deviation for south east asians
sas["no_strat_no_causal"] = np.random.normal(mu, sigma, len(sas))  # 5
sas["bias_no_strat"] = 150

# Make synthetic phenotype with population stratification and no causal effect

mu, sigma = 190, sigmap  # mean and standard deviation for africans
afr["strat_no_causal"] = np.random.normal(mu, sigma, len(afr))
afr["bias_strat"] = 180
mu, sigma = 170, sigmap  # mean and standard deviation for americans
amr["strat_no_causal"] = np.random.normal(mu, sigma, len(amr))
amr["bias_strat"] = 170
mu, sigma = 150, sigmap  # mean and standard deviation for east asians
eas["strat_no_causal"] = np.random.normal(mu, sigma, len(eas))
eas["bias_strat"] = 150
mu, sigma = 180, sigmap  # mean and standard deviation for europeans
eur["strat_no_causal"] = np.random.normal(mu, sigma, len(eur))  # 4
eur["bias_strat"] = 175
mu, sigma = 160, sigmap  # mean and standard deviation for south east asians
sas["strat_no_causal"] = np.random.normal(mu, sigma, len(sas))  # 5
sas["bias_strat"] = 160
humans = pd.concat([afr, amr, eas, eur, sas]).sort_index()


/tmp/ipykernel_2489/3265141721.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afr["no_strat_no_causal"] = np.random.normal(mu, sigma, len(afr))
/tmp/ipykernel_2489/3265141721.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afr["bias_no_strat"] = 150
/tmp/ipykernel_2489/3265141721.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [360]:
humans['no_strat_causal'] = humans['no_strat_no_causal']
humans['strat_causal'] = humans['strat_no_causal']


path_genos = PATH_GENO_segmented + f"chrom_{chrom}/unshuffled/"
onlyfiles_input = [f for f in listdir(path_genos) if float(f.split('_')[5]) >= 0.1]

In [361]:
onlyfiles_input

['17_0_1200_17_maf_0.12_0.16.pkl',
 '17_1_1200_17_maf_0.12_0.16.pkl',
 '17_2_1147_17_maf_0.12_0.16.pkl',
 '17_3_800_17_maf_0.12_0.16.pkl',
 '17_4_1174_17_maf_0.12_0.16.pkl',
 '17_5_879_17_maf_0.12_0.16.pkl',
 '17_6_800_17_maf_0.12_0.16.pkl',
 '17_7_1200_17_maf_0.12_0.16.pkl',
 '17_8_800_17_maf_0.12_0.16.pkl',
 '17_9_800_17_maf_0.12_0.16.pkl',
 '20_0_1200_20_maf_0.25_0.31.pkl',
 '20_1_800_20_maf_0.25_0.31.pkl',
 '20_2_966_20_maf_0.25_0.31.pkl',
 '20_3_800_20_maf_0.25_0.31.pkl',
 '20_4_1090_20_maf_0.25_0.31.pkl',
 '20_5_800_20_maf_0.25_0.31.pkl',
 '20_6_836_20_maf_0.25_0.31.pkl',
 '20_7_1200_20_maf_0.25_0.31.pkl',
 '20_8_1108_20_maf_0.25_0.31.pkl',
 '20_9_1200_20_maf_0.25_0.31.pkl',
 '22_0_1200_22_maf_0.37_0.43.pkl',
 '22_1_996_22_maf_0.37_0.43.pkl',
 '22_2_800_22_maf_0.37_0.43.pkl',
 '22_3_1200_22_maf_0.37_0.43.pkl',
 '22_4_995_22_maf_0.37_0.43.pkl',
 '22_5_800_22_maf_0.37_0.43.pkl',
 '22_6_800_22_maf_0.37_0.43.pkl',
 '22_7_1200_22_maf_0.37_0.43.pkl',
 '22_8_809_22_maf_0.37_0.43.pkl',
 

In [362]:
nums = [f.split('_')[0] for f in onlyfiles_input]
uniq = list(Counter(nums).keys())
snps_popped = []
mafs = []
causal_effects = []
for num in uniq[0:3]:
    segment = [f for f in onlyfiles_input if f.split('_')[0] == num]
    popped = segment.pop()
    maf = popped.split("maf_")[1].split('.pkl')[0]
    mafs.append(maf)
    snps = pd.read_pickle(path_genos+popped)
    snp_nr = 1
    snp = snps.iloc[:, snp_nr]
    snps_popped.append(snp)
    name = snps.columns[snp_nr]
    afs = popped.split('maf_')[1].split('.pkl')[0]
    causal_effects.append(effect)
    snp_effect = snps.iloc[:, snp_nr] * effect
    #snp_effect = snps.iloc[:, snp_nr] * (effect/float(maf.split('_')[0]))

    humans['no_strat_causal'] = humans['no_strat_causal'] + snp_effect
    humans['strat_causal'] = humans['strat_causal'] + snp_effect

In [363]:
print(Counter(snp_effect).keys())


dict_keys([0, 1, -1])


In [364]:
causal_snps = pd.DataFrame(data=snps_popped).T
mafs = pd.DataFrame(data=mafs)
c_effects = pd.DataFrame(data=causal_effects)

In [365]:
Cs = list(causal_snps.columns)
Cs = pd.DataFrame(data=Cs, columns=['snp'])
Cs['effects'] = list(c_effects[0])
Cs['mafs'] = list(mafs[0])

In [366]:
path_output = PATH_GENO + "/sumstat/"

In [367]:
sit = "strat_causal"

In [368]:
bias = np.array(humans[sit])

In [369]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [370]:
for file in onlyfiles_input:
    path_file = path_genos + '/' + file
    genos = pd.read_pickle(path_file)
    #AF = pd.read_pickle(path_mafs + file)
    #effect_matrix = genos - AF
    effect_matrix = genos
    X = effect_matrix
    y = bias
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    sumstat = pd.DataFrame(data={
        'snp': list(est2.pvalues.index)[1:],
        'coef': list(est2.params)[1:],
        'std_err': list(est2.bse)[1:],
        't': list(est2.tvalues)[1:],
        'P': list(est2.pvalues[1:]),
        '0.025': list(est2.conf_int()[0])[1:],
        '0.975': list(est2.conf_int()[1])[1:]
    })
    sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
    sumstat['POS'] = [i.split(':', 1)[0]
                      for i in list(sumstat['snp'].str[3:-5])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    sumstat['-logp'] = - np.log10(sumstat['P'])
    print(sumstat)
    sumstat.to_pickle(f"{path_output}chrom_{chrom}/{file}")


                    snp      coef    std_err         t         P      0.025  \
0     21:16998015:A:G_G -6.958811   3.982614 -1.747297  0.080808 -14.771423   
1     21:43201378:A:G_A  3.949435   3.451545  1.144251  0.252717  -2.821390   
2     21:39830556:C:T_C -0.443690   1.212844 -0.365826  0.714551  -2.822901   
3     21:43867844:A:G_G  0.178846   0.379069  0.471803  0.637142  -0.564766   
4     21:41253722:G:T_G  4.603434   3.656213  1.259072  0.208217  -2.568884   
...                 ...       ...        ...       ...       ...        ...   
1195  21:15318511:C:G_G  3.107245   1.378643  2.253843  0.024362   0.402789   
1196  21:43590282:A:T_A  5.287345   2.940067  1.798375  0.072335  -0.480125   
1197  21:46400789:C:T_C  1.769085  10.427865  0.169650  0.865310 -18.687044   
1198  21:43592576:A:C_A  5.287345   2.940067  1.798375  0.072335  -0.480125   
1199  21:27581794:C:T_C  2.151450   2.112160  1.018602  0.308570  -1.991931   

          0.975  #CHROM       POS     -logp  
0    

In [371]:
path_sumstat = PATH_SUMSTAT

chrom_files = [f for f in listdir(path_sumstat) if f != 'total_sumstat.pkl']
chrom_files = [f for f in chrom_files if f == f"chrom_{chrom}"]
sumstat_array = []
for chrom_file in chrom_files:
    sumstat_files = [f for f in listdir(path_sumstat + chrom_file + '/')]
    # first do it with the first one
    path_file = path_sumstat + chrom_file + '/' + sumstat_files[0]
    sumstats = pd.read_pickle(path_file)
    for sumstat_file in sumstat_files[1:]:
        path_file = path_sumstat + chrom_file + '/' + sumstat_file
        temp = pd.read_pickle(path_file)
        sumstats = pd.concat([sumstats,temp])
    sumstat_array.append(sumstats)

sumstats = sumstat_array[0]
for sumstat in sumstat_array[1:]:
    sumstats = pd.concat([sumstats,sumstat]) 

In [372]:
sumstats = sumstats.reset_index(drop=True)


In [373]:
running_pos = 0
cumulative_pos = []
for chrom, group_df in sumstats.groupby('#CHROM'):
    cumulative_pos.append(group_df['POS'] + running_pos)
    running_pos += group_df['POS'].max()

sumstats['cumulative_pos'] = pd.concat(cumulative_pos)
sumstats['SNP number'] = sumstats.index


In [374]:
sns.relplot(
    data=sumstats,
    x='cumulative_pos',
    y='-logp',
    aspect=2,
    hue='#CHROM',
    palette='dark'
).set(title=f"test")
plt.savefig(f"./GWAS_Common.png")
